# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [4]:
# Importing the dataset
pokemon = pd.read_csv("pokemon.csv")
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [9]:
# your code here
#What is the formula to compute Total?
#Total='HP'+'Attack'+'Defense'+'Sp.Atk'+'Sp.Def'+'Speed'

#Does the formula work for all pokemon? To check
pokemon['Totel_test']=pokemon['HP']+pokemon['Attack']+pokemon['Defense']+pokemon['Sp. Atk']+pokemon['Sp. Def']+pokemon['Speed']

#To check
pokemon['Total']==pokemon['Totel_test']

0      True
1      True
2      True
3      True
4      True
       ... 
795    True
796    True
797    True
798    True
799    True
Length: 800, dtype: bool

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [33]:
# your code here
from sklearn.preprocessing import OneHotEncoder
from seaborn import load_dataset
from sklearn.compose import make_column_transformer

type1 = pokemon.select_dtypes(include='object')

type1 = type1.drop(['Type 2'], axis=1)
type1
encoder1 = OneHotEncoder().fit(type1) 

cols1 = encoder1.get_feature_names_out(input_features=type1.columns)

type1_encode = pd.DataFrame(encoder1.transform(type1).toarray(),columns=cols1)

type1_encode.head()
type2 = pokemon.select_dtypes(include='object')

type2 = type2.drop(['Type 1'], axis=1)
type2
encoder2 = OneHotEncoder().fit(type2) 

cols2 = encoder2.get_feature_names_out(input_features=type2.columns)

type2_encode = pd.DataFrame(encoder2.transform(type2).toarray(),columns=cols2)

type2_encode.head()
type_encode = pd.concat([type1_encode, type2_encode], axis=1)
type_encode
def add_type_columns(df):
    types = ['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire', 'Flying',
             'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison', 'Psychic', 'Rock', 'Steel', 'Water']
    for t in types:
        type_1_col = f'Type 1_{t}'
        type_2_col = f'Type 2_{t}'
        if type_1_col in df.columns and type_2_col in df.columns:
            df[f'Type_{t}'] = df[type_1_col] + df[type_2_col]
            df.drop([type_1_col, type_2_col], axis=1, inplace=True) 
    return df
type_encode1 = add_type_columns(type_encode)
type_encode1

,Name_Abomasnow,Name_AbomasnowMega Abomasnow,Name_Abra,Name_Absol,Name_AbsolMega Absol,Name_Accelgor,Name_AegislashBlade Forme,Name_AegislashShield Forme,Name_Aerodactyl,Name_AerodactylMega Aerodactyl,...,Type_Ghost,Type_Grass,Type_Ground,Type_Ice,Type_Normal,Type_Poison,Type_Psychic,Type_Rock,Type_Steel,Type_Water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
798,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [40]:
# your code here
type_encode1['Total'] = pokemon['Totel_test']
type_encode1
import matplotlib.pyplot as plt
import seaborn as sns

correlations_matrix = type_encode1.corr()
display(correlations_matrix)

correlations = type_encode1.corr()
correlations_sorted = correlations['Total'].sort_values(ascending=False)
print(correlations_sorted.head(5))

,Name_Abomasnow,Name_AbomasnowMega Abomasnow,Name_Abra,Name_Absol,Name_AbsolMega Absol,Name_Accelgor,Name_AegislashBlade Forme,Name_AegislashShield Forme,Name_Aerodactyl,Name_AerodactylMega Aerodactyl,...,Type_Grass,Type_Ground,Type_Ice,Type_Normal,Type_Poison,Type_Psychic,Type_Rock,Type_Steel,Type_Water,Total
Name_Abomasnow,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,0.096374,-0.010696,0.158421,-0.013524,-0.010254,-0.012596,-0.009891,-0.009037,-0.015296,0.017380
Name_AbomasnowMega Abomasnow,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,0.096374,-0.010696,0.158421,-0.013524,-0.010254,-0.012596,-0.009891,-0.009037,-0.015296,0.046889
Name_Abra,-0.001252,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.012987,-0.010696,-0.007900,-0.013524,-0.010254,0.099365,-0.009891,-0.009037,-0.015296,-0.036916
Name_Absol,-0.001252,-0.001252,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.012987,-0.010696,-0.007900,-0.013524,-0.010254,-0.012596,-0.009891,-0.009037,-0.015296,0.008822
Name_AbsolMega Absol,-0.001252,-0.001252,-0.001252,-0.001252,1.000000,-0.001252,-0.001252,-0.001252,-0.001252,-0.001252,...,-0.012987,-0.010696,-0.007900,-0.013524,-0.010254,-0.012596,-0.009891,-0.009037,-0.015296,0.038331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Type_Psychic,-0.012596,-0.012596,0.099365,-0.012596,-0.012596,-0.012596,-0.012596,-0.012596,-0.012596,-0.012596,...,-0.094009,-0.079079,-0.042311,-0.112381,-0.103195,1.000000,-0.069031,0.024540,-0.099639,0.124688
Type_Rock,-0.009891,-0.009891,-0.009891,-0.009891,-0.009891,-0.009891,-0.009891,-0.009891,0.126536,0.126536,...,-0.072829,0.072085,-0.017110,-0.106877,-0.081036,-0.069031,1.000000,0.049202,0.011447,0.032731
Type_Steel,-0.009037,-0.009037,-0.009037,-0.009037,-0.009037,-0.009037,0.138500,0.138500,-0.009037,-0.009037,...,-0.061537,-0.020771,-0.057042,-0.097645,-0.074036,0.024540,0.049202,1.000000,-0.096131,0.109703
Type_Water,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,-0.015296,...,-0.126894,-0.006844,0.000242,-0.154994,-0.073986,-0.099639,0.011447,-0.096131,1.000000,-0.021665


Total                         1.000000
Type_Dragon                   0.229705
Type_Psychic                  0.124688
Type_Steel                    0.109703
Name_RayquazaMega Rayquaza    0.101775
Name: Total, dtype: float64


# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# your code here